# 📚 Institutional Books Dataset Loader

Load books from HuggingFace **institutional/institutional-books-1.0** dataset into BookLook database.

## Workflow
1. Load data from dataset (streaming)
2. Structure with pandas DataFrame
3. Rename fields to match database schema
4. Push to database in batches

## Field Mapping
Dataset → Database:
- `title_src` → `titre`
- `author_src` → `author_names` (array)
- `date1_src` → `date_publication`
- `page_count_src` → `nombre_pages`, `total_pages`
- `language_gen` → `langue`
- `general_note_src` → `description`
- `token_count_o200k_base_gen` → `word_count` (×0.75)
- `genre_or_form_src` + `topic_or_subject_gen` → `genre_names` (array)
- `identifiers_src.isbn` → `isbn`

## Step 1: Install Dependencies

In [ ]:
!pip install datasets huggingface-hub psycopg2-binary requests pandas -q

## Step 2: Import Libraries

In [ ]:
import os
import json
import time
import re
from typing import List, Dict, Optional
from datetime import datetime

import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import requests
from datasets import load_dataset
from huggingface_hub import login

print("✅ All imports successful!")

## Step 3: Configuration (⚠️ UPDATE YOUR TOKEN)

In [ ]:
# ⚠️ UPDATE THIS: Your HuggingFace token
HF_TOKEN = "YOUR_HUGGINGFACE_TOKEN_HERE"

# Dataset configuration
DATASET_NAME = "institutional/institutional-books-1.0"
CHUNK_SIZE = 100  # Books per batch
MAX_CHUNKS = 10   # Set to None for all books

# Database configuration
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'book_library',
    'user': 'bookuser',
    'password': 'bookpass123'
}

# Progress tracking
PROGRESS_FILE = 'load_progress.json'

print(f"📚 Dataset: {DATASET_NAME}")
print(f"📦 Chunk size: {CHUNK_SIZE}")
print(f"🔢 Max chunks: {MAX_CHUNKS if MAX_CHUNKS else 'All'}")

## Step 4: Helper Functions

In [ ]:
def extract_isbn(identifiers_src: Dict) -> str:
    """Extract ISBN from identifiers_src field."""
    if not identifiers_src or 'isbn' not in identifiers_src:
        return None
    isbns = identifiers_src['isbn']
    if isinstance(isbns, list) and len(isbns) > 0:
        isbn = str(isbns[0]).strip().replace('-', '').replace(' ', '')
        if len(isbn) in [10, 13]:
            return isbn
    return None

def generate_isbn_from_barcode(barcode: str) -> str:
    """Generate pseudo-ISBN from barcode."""
    barcode_hash = abs(hash(barcode)) % (10 ** 10)
    return f"999{barcode_hash:010d}"

def parse_date(date_str: str) -> Optional[str]:
    """Parse date1_src to YYYY-MM-DD format."""
    if not date_str:
        return None
    year_match = re.search(r'\b(1[0-9]{3}|20[0-9]{2})\b', str(date_str))
    if year_match:
        return f"{year_match.group(1)}-01-01"
    return None

def clean_text(text: str, max_length: int = 2000) -> str:
    """Clean and truncate text."""
    if not text:
        return ""
    text = re.sub(r'\s+', ' ', text).strip()
    if len(text) > max_length:
        text = text[:max_length] + "..."
    return text

def map_language(lang_code: str) -> str:
    """Map ISO 639-3 to full language name."""
    lang_map = {
        'eng': 'English', 'fra': 'French', 'deu': 'German',
        'spa': 'Spanish', 'ita': 'Italian', 'por': 'Portuguese',
        'rus': 'Russian', 'jpn': 'Japanese', 'chi': 'Chinese'
    }
    return lang_map.get(lang_code[:3].lower(), lang_code or 'English')

def extract_genres(genre_or_form_src: str, topic_or_subject_gen: str) -> List[str]:
    """Extract genres from dataset fields."""
    genres = []
    if genre_or_form_src:
        parts = re.split(r'[;,|]', genre_or_form_src)
        genres.extend([g.strip() for g in parts if g.strip()])
    if topic_or_subject_gen and topic_or_subject_gen not in genres:
        genres.append(topic_or_subject_gen)
    return genres[:3] if genres else ['General']

def fetch_cover_image(isbn: str) -> Optional[str]:
    """Fetch cover image from Open Library."""
    if not isbn or isbn.startswith('999'):
        return None
    try:
        url = f"https://covers.openlibrary.org/b/isbn/{isbn}-L.jpg"
        response = requests.head(url, timeout=2)
        if response.status_code == 200:
            return url
    except:
        pass
    return None

print("✅ Helper functions defined")

## Step 5: Progress Tracking

In [ ]:
def load_progress() -> Dict:
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            return json.load(f)
    return {'last_index': -1, 'total_loaded': 0, 'timestamp': None}

def save_progress(index: int, total: int):
    with open(PROGRESS_FILE, 'w') as f:
        json.dump({
            'last_index': index,
            'total_loaded': total,
            'timestamp': datetime.now().isoformat()
        }, f, indent=2)

print("✅ Progress tracking defined")

## Step 6: Login to HuggingFace

In [ ]:
login(token=HF_TOKEN)
print("✅ Logged in to HuggingFace")

## Step 7: Load Dataset with Streaming

In [ ]:
progress = load_progress()
start_index = progress['last_index'] + 1

if start_index > 0:
    print(f"📍 Resuming from index {start_index}")

print(f"📥 Loading dataset: {DATASET_NAME}")
dataset_stream = load_dataset(DATASET_NAME, split="train", streaming=True)
print("✅ Dataset loaded")

## Step 8: Connect to Database

In [ ]:
conn = psycopg2.connect(**DB_CONFIG)
cursor = conn.cursor()
print("✅ Database connected")

## Step 9: Process Data with Pandas

This cell:
1. Collects books into chunks
2. Creates pandas DataFrame with dataset fields
3. Transforms and renames to database schema
4. Inserts into database

In [ ]:
print(f"\n🚀 Starting data loading")
print("=" * 80)

chunk_buffer = []
current_index = 0
chunk_number = 0
total_inserted = progress['total_loaded']
total_skipped = 0
start_time = time.time()

try:
    for book_data in dataset_stream:
        if current_index < start_index:
            current_index += 1
            continue
        
        chunk_buffer.append(book_data)
        current_index += 1
        
        if len(chunk_buffer) >= CHUNK_SIZE:
            chunk_number += 1
            print(f"\n📦 Chunk {chunk_number} (books {current_index - CHUNK_SIZE + 1}-{current_index})")
            print("-" * 80)
            
            # Create DataFrame from chunk
            df = pd.DataFrame(chunk_buffer)
            print(f"   📊 DataFrame created: {len(df)} rows")
            
            # Transform data
            df['isbn'] = df.apply(lambda row: extract_isbn(row.get('identifiers_src')) or generate_isbn_from_barcode(row.get('barcode_src', '')), axis=1)
            df['titre'] = df['title_src'].fillna('Unknown Title')
            df['author_names'] = df['author_src'].apply(lambda x: [x] if x else ['Unknown Author'])
            df['date_publication'] = df['date1_src'].apply(parse_date)
            df['description'] = df['general_note_src'].apply(lambda x: clean_text(x) if x else 'No description')
            df['nombre_pages'] = df['page_count_src'].fillna(0).astype(int)
            df['total_pages'] = df['nombre_pages']
            df['langue'] = df['language_gen'].fillna(df['language_src']).apply(map_language)
            df['word_count'] = (df['token_count_o200k_base_gen'].fillna(0) * 0.75).astype(int)
            df['genre_names'] = df.apply(lambda row: extract_genres(row.get('genre_or_form_src'), row.get('topic_or_subject_gen')), axis=1)
            df['image_url'] = df['isbn'].apply(fetch_cover_image)
            
            # Select and rename columns for database
            db_df = df[[
                'isbn', 'titre', 'date_publication', 'description', 'image_url',
                'nombre_pages', 'total_pages', 'langue', 'word_count',
                'author_names', 'genre_names'
            ]].copy()
            
            # Remove duplicates by ISBN
            db_df = db_df.drop_duplicates(subset=['isbn'], keep='first')
            
            # Check existing ISBNs
            existing_isbns = set()
            cursor.execute("SELECT isbn FROM books WHERE isbn = ANY(%s)", (db_df['isbn'].tolist(),))
            existing_isbns = {row[0] for row in cursor.fetchall()}
            
            # Filter out existing
            db_df = db_df[~db_df['isbn'].isin(existing_isbns)]
            
            if len(db_df) == 0:
                print(f"   ⏭️  All {len(chunk_buffer)} books already exist")
                total_skipped += len(chunk_buffer)
            else:
                # Insert books
                insert_query = """
                    INSERT INTO books (
                        isbn, titre, date_publication, description, image_url,
                        nombre_pages, total_pages, langue, word_count,
                        author_names, genre_names,
                        note_moyenne, nombre_reviews, average_rating, review_count,
                        created_at
                    ) VALUES %s
                    ON CONFLICT (isbn) DO NOTHING
                """
                
                values = [
                    (
                        row['isbn'], row['titre'], row['date_publication'],
                        row['description'], row['image_url'],
                        row['nombre_pages'], row['total_pages'], row['langue'],
                        row['word_count'], row['author_names'], row['genre_names'],
                        0.0, 0, 0.0, 0, datetime.now()
                    )
                    for _, row in db_df.iterrows()
                ]
                
                execute_values(cursor, insert_query, values)
                conn.commit()
                
                inserted = len(db_df)
                skipped = len(chunk_buffer) - inserted
                total_inserted += inserted
                total_skipped += skipped
                
                print(f"   ✅ Inserted: {inserted}")
                print(f"   ⏭️  Skipped: {skipped}")
            
            # Save progress
            save_progress(current_index - 1, total_inserted)
            
            elapsed = time.time() - start_time
            rate = total_inserted / elapsed if elapsed > 0 else 0
            print(f"   📊 Total: {total_inserted} loaded, {total_skipped} skipped")
            print(f"   ⏱️  Rate: {rate:.1f} books/sec")
            
            chunk_buffer = []
            
            if MAX_CHUNKS and chunk_number >= MAX_CHUNKS:
                print(f"\n🏁 Reached max chunks ({MAX_CHUNKS})")
                break

except KeyboardInterrupt:
    print("\n⚠️  Interrupted")
    conn.commit()
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
    conn.rollback()

elapsed = time.time() - start_time
print("\n" + "=" * 80)
print("✅ LOADING COMPLETE")
print("=" * 80)
print(f"📊 Total inserted: {total_inserted}")
print(f"⏭️  Total skipped: {total_skipped}")
print(f"📦 Chunks: {chunk_number}")
print(f"⏱️  Time: {elapsed:.1f}s")
print(f"📈 Rate: {total_inserted / elapsed:.1f} books/sec")
print("=" * 80)

## Step 10: Close Connection

In [ ]:
cursor.close()
conn.close()
print("🔌 Database closed")

## Step 11: View Summary

In [ ]:
conn = psycopg2.connect(**DB_CONFIG)
cursor = conn.cursor()

print("\n📊 Database Summary:")
print("=" * 80)

cursor.execute("SELECT COUNT(*) FROM books")
print(f"📚 Total Books: {cursor.fetchone()[0]}")

cursor.execute("SELECT COUNT(*) FROM books WHERE image_url IS NOT NULL")
print(f"🖼️  Books with covers: {cursor.fetchone()[0]}")

cursor.execute("SELECT langue, COUNT(*) FROM books GROUP BY langue ORDER BY COUNT(*) DESC LIMIT 5")
print("\n🌍 Top 5 Languages:")
for lang, count in cursor.fetchall():
    print(f"   {lang}: {count} books")

cursor.close()
conn.close()
print("\n✅ Summary complete!")